In [6]:
import pandas as pd
import numpy as np
import joblib
import uvicorn
import pickle
from fastapi import FastAPI, HTTPException
from fastapi.middleware.cors import CORSMiddleware
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import random
from pydantic import BaseModel
import nest_asyncio
from statsmodels.tsa.arima.model import ARIMA
from datetime import datetime, timedelta
from pyngrok import ngrok

# Ngrok Authentication
NGROK_AUTH_TOKEN = "28WOZmFsePzDohg4URD5MU7tgBN_3onpraLYjUsXNXoB4iiro"
ALLOWED_ORIGINS = ["https://disease-surveillance-ai-new.vercel.app", "http://localhost:3000"]

ngrok.set_auth_token(NGROK_AUTH_TOKEN)
app = FastAPI()

# Enable CORS
app.add_middleware(
    CORSMiddleware,
    allow_origins=ALLOWED_ORIGINS,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

#-------------------------------
# Module 1 :- Disease Trend Analysis (Disease Case Forecasting)
#-------------------------------

# Load your trained ARIMA model (ensure it's saved properly)
with open("/content/drive/MyDrive/SSIP 2025 - AI-Powered Predictive Analytics for Disease Surveillance & Outbreak Management/Disease Trend Analysis (Case Forecasting)/Disease Trend Analysis Model/trained_arima_model.pkl", "rb") as f:
    trained_model = pickle.load(f)

# Request model: Only needs today's cases
class CaseInput(BaseModel):
    today_cases: int

@app.post("/disease_forecast/predict")
def get_forecast(data: CaseInput):
    """
    Predicts the next 7 days using the trained ARIMA model.
    """
    today_cases = data.today_cases
    days = 7  # Fixed 7-day forecast

    # Use the trained ARIMA model to forecast
    forecast_values = trained_model.forecast(steps=days)

    # Adjust predictions based on today's cases (optional)
    forecast_values = [today_cases + (val - forecast_values[0]) for val in forecast_values]

    # Generate future dates
    future_dates = [(datetime.today() + timedelta(days=i)).strftime('%Y-%m-%d') for i in range(1, days + 1)]

    # Prepare JSON response
    forecast_data = [{"date": date, "predicted_cases": round(value)} for date, value in zip(future_dates, forecast_values)]

    return {"forecast": forecast_data}



#-------------------------------
# Module 2 :- Disease Outbreak Prediction
#-------------------------------
# Load the Trained Model & Scaler
xgboost_model_filename = "/content/drive/MyDrive/SSIP 2025 - AI-Powered Predictive Analytics for Disease Surveillance & Outbreak Management/Disease Outbreak Prediction/xgboost_model.pkl"
scaler_filename = "/content/drive/MyDrive/SSIP 2025 - AI-Powered Predictive Analytics for Disease Surveillance & Outbreak Management/Disease Outbreak Prediction/scaler.pkl"

xgboost_model = joblib.load(xgboost_model_filename)
scaler = joblib.load(scaler_filename)
expected_features = scaler.feature_names_in_

# Input Data Model
class InputData(BaseModel):
    total_cases: int
    active_cases: int
    mild_cases: int
    moderate_cases: int
    severe_cases: int
    mortality_rate: float
    recovery_rate: float
    occupied_beds: int
    occupied_ventilators: int
    occupied_oxygen: int
    total_icu_beds: int
    icu_utilization: int
    emergency_admission_rate: int
    resource_load: int

# Risk Classification
def classify_outbreak_risk(prediction):
    historical_outbreak_values = np.random.randint(50, 5000, size=100)
    low_threshold = np.percentile(historical_outbreak_values, 25)
    moderate_threshold = np.percentile(historical_outbreak_values, 50)
    high_threshold = np.percentile(historical_outbreak_values, 75)

    if prediction < low_threshold:
        return "Low Risk"
    elif prediction < moderate_threshold:
        return "Moderate Risk"
    elif prediction < high_threshold:
        return "High Risk"
    else:
        return "Critical Risk"

# Prediction Endpoint
@app.post("/disease_outbreak/predict")
def predict_outbreak(data: InputData):
    try:
        # Convert input to DataFrame
        new_data = pd.DataFrame([data.model_dump()])  # ✅ Fix 1: Use model_dump() instead of dict()

        # Feature Renaming
        feature_mapping = {
            "total_cases": "Total Cases",
            "active_cases": "Active Cases",
            "mild_cases": "Mild Cases",
            "moderate_cases": "Moderate Cases",
            "severe_cases": "Severe Cases",
            "mortality_rate": "Mortality Rate",
            "recovery_rate": "Recovery Rate",
            "occupied_beds": "Occupied Beds",
            "occupied_ventilators": "Occupied Ventilators",
            "occupied_oxygen": "Occupied Oxygen",
            "total_icu_beds": "Total ICU Beds in Hospital",
            "icu_utilization": "ICU Utilization",
            "emergency_admission_rate": "Emergency Admission Rate",
            "resource_load": "Resource Load"
        }
        new_data.rename(columns=feature_mapping, inplace=True)
        new_data = new_data[expected_features]

        # Scale Input
        new_data_scaled = scaler.transform(new_data)

        # XGBoost Prediction
        xgb_prediction = xgboost_model.predict(new_data_scaled)[0]

        # ✅ Fix 2: Convert to standard Python float
        prediction_value = float(xgb_prediction)

        # Get Risk Category
        risk_category = classify_outbreak_risk(prediction_value)

        return {
            "prediction": prediction_value,
            "risk_category": risk_category
        }
    except Exception as e:
        raise HTTPException(status_code=400, detail=str(e))


#-------------------------------
# Module 3 :- Symptoms Based Disease Classification
#-------------------------------

# Load the trained model & label encoder
model = joblib.load("/content/drive/MyDrive/SSIP 2025 - AI-Powered Predictive Analytics for Disease Surveillance & Outbreak Management/Symptoms Based Disease Classification/disease_model.pkl")
label_encoder = joblib.load("/content/drive/MyDrive/SSIP 2025 - AI-Powered Predictive Analytics for Disease Surveillance & Outbreak Management/Symptoms Based Disease Classification/label_encoder.pkl")
dataset_path = "/content/drive/MyDrive/SSIP 2025 - AI-Powered Predictive Analytics for Disease Surveillance & Outbreak Management/Symptoms Based Disease Classification/Training.csv"

# Load training data to extract symptom columns
train_df = pd.read_csv(dataset_path)
train_df = train_df.drop(columns=["Unnamed: 133"], errors="ignore")

# Extract feature names (symptoms)
symptom_columns = list(train_df.columns[:-1])


# Request body format
class SymptomInput(BaseModel):
    symptoms: list

@app.post("/disease_classify/predict")
def predict_disease(input_data: SymptomInput):
    """
    Predicts the disease based on input symptoms.

    Args:
        symptoms (list): List of symptoms (e.g., ["itching", "skin_rash"])

    Returns:
        str: Predicted disease name
    """
    symptoms_vector = np.zeros((1, len(symptom_columns)))

    for symptom in input_data.symptoms:
        if symptom in symptom_columns:
            symptoms_vector[0][symptom_columns.index(symptom)] = 1

    prediction = model.predict(symptoms_vector)
    predicted_disease = label_encoder.inverse_transform(prediction)[0]

    return {"predicted_disease": predicted_disease}

#-------------------------------
# Module 4 :- Medical Resource Allocation
#-------------------------------

#-------------------------------
# Module 5 :- Social Media And News API
#-------------------------------
def load_model(model_name="distilgpt2"):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)
    return tokenizer, model

districts = [
    "Ahmedabad", "Amreli", "Anand", "Aravalli", "Banaskantha", "Bharuch", "Bhavnagar", "Botad", "Chhota Udaipur", "Dahod",
    "Dang", "Devbhoomi Dwarka", "Gandhinagar", "Gir Somnath", "Jamnagar", "Junagadh", "Kheda", "Kutch", "Mahisagar", "Mehsana",
    "Morbi", "Narmada", "Navsari", "Panchmahal", "Patan", "Porbandar", "Rajkot", "Sabarkantha", "Surat", "Surendranagar",
    "Tapi", "Vadodara", "Valsad"
]

villages = [
    "Dholka", "Viramgam", "Maliya", "Dediapada", "Mandvi", "Dhrol", "Bagasara", "Halol", "Kalol", "Lathi", "Rapar", "Babra", "Una"
]

diseases = ["Dengue", "Malaria", "Chikungunya", "Swine Flu", "Typhoid", "Hepatitis", "COVID-19", "Zika Virus", "Tuberculosis", "Cholera"]

def generate_text(prompt, tokenizer, model, max_length=150):
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(**inputs, max_length=max_length, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id)
    text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return text.strip().replace("\n", " ")  # Remove unnecessary newlines and extra spaces

def generate_news():
    tokenizer, model = load_model()
    disease = random.choice(diseases)
    location = random.choice(districts + villages)
    news_templates = [
        f"🚨 Breaking News: {disease} outbreak reported in {location}. Health officials urge immediate action.",
        f"⚠️ Health Alert: {location} sees a sharp rise in {disease} cases. Hospitals are preparing for more patients.",
        f"📢 Government Notice: Authorities in {location} impose new health restrictions due to rising {disease} cases.",
        f"🦠 Medical Update: {location} records its highest {disease} cases this month. Doctors warn of potential complications.",
        f"📊 Statistics Report: {disease} cases in {location} have increased by {random.randint}% in the last week. Citizens urged to stay cautious.",
        f"🔬 Research Findings: Scientists in {location} study {disease} spread patterns to prevent future outbreaks.",
        f"🏥 Healthcare Update: Hospitals in {location} report increased admissions due to {disease}. Medical resources are under strain.",
        f"🌍 Global Concern: {disease} cases in {location} spark concerns of possible regional outbreak. Experts recommend immediate measures.",
        f"📰 Local News: {location} witnesses unusual {disease} spike. Public urged to follow preventive guidelines strictly."
    ]
    prompt = random.choice(news_templates)
    news_article = generate_text(prompt, tokenizer, model, max_length=150)
    return news_article

def generate_social_media_post():
    tokenizer, model = load_model()
    disease = random.choice(diseases)
    location = random.choice(districts)
    social_media_templates = [
        f"🚨 Stay Safe! {disease} cases are rising in {location}. Follow safety measures and avoid crowded places! #HealthAlert",
        f"⚠️ Attention {location} residents! {disease} infections are increasing. Wash your hands regularly and stay protected!",
        f"🦠 {disease} outbreak in {location}. Health officials recommend vaccination and hygiene measures to stop the spread.",
        f"📢 Urgent! {disease} cases in {location} have surged. Wear masks and maintain social distancing! #StaySafe",
        f"🔬 Researchers in {location} are working on a new treatment for {disease}. Stay tuned for more updates!",
        f"🏥 Hospitals in {location} are seeing an increase in {disease} cases. Please follow medical guidelines to protect yourself.",
        f"📊 The latest data shows a rise in {disease} cases in {location}. Be cautious and stay informed!",
        f"🌍 {location} is facing a growing {disease} crisis. Authorities urge people to take all necessary precautions.",
        f"📰 Local Update: {location} sees an increase in {disease} cases. Help stop the spread by following health guidelines!"
    ]
    prompt = random.choice(social_media_templates)
    social_post = generate_text(prompt, tokenizer, model, max_length=150)
    return social_post

@app.get("/disease_news/predict")
def get_news():
    return {generate_news()}

@app.get("/disease_social/predict")
def get_social_media_posts():
    return {generate_social_media_post()}

#-------------------------------
# Module 6 :- Automated Alerts and Notifications
#-------------------------------

# Run FastAPI with Ngrok
if __name__ == "__main__":
    nest_asyncio.apply()  # Allows nested event loops in Jupyter environment
    public_url = ngrok.connect(8000).public_url
    print(f"FastAPI running on: {public_url}")
    uvicorn.run(app, host="0.0.0.0", port=8000)

ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-12' coro=<Server.serve() done, defined at /usr/local/lib/python3.11/dist-packages/uvicorn/server.py:68> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/main.py", line 579, in run
    server.run()
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/server.py", line 66, in run
    return asyncio.run(self.serve(sockets=sockets))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 92, in run_until_complete
    self._run_once()
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 133, in _run_once
    handle._run()
  File "/usr/lib/python3.11/asyncio/events.py", line 84, in _run
    s

FastAPI running on: https://a39f-34-48-155-57.ngrok-free.app


INFO:     Started server process [262]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     2409:40c1:1a:a8cd:3163:f88d:720c:bea8:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     2409:40c1:1a:a8cd:3163:f88d:720c:bea8:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     2409:40c1:1a:a8cd:3163:f88d:720c:bea8:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     2409:40c1:1a:a8cd:3163:f88d:720c:bea8:0 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     2409:40c1:1a:a8cd:3163:f88d:720c:bea8:0 - "GET /disease_news/predict HTTP/1.1" 200 OK
INFO:     2409:40c1:1a:a8cd:3163:f88d:720c:bea8:0 - "GET /disease_social/predict HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [262]
